In [1]:
import IPython

import pandas as pd
import matplotlib.pyplot as plt
import qiime2 as q2
import seaborn as sns
from qiime2 import Visualization


import os

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
PATH = "livia_data/fungut_metadata.tsv"

In [3]:
surveys_df = pd.read_csv(PATH, sep="\t")

In [4]:
data_dir = 'livia_data'

In [5]:
!qiime tools peek $data_dir/fungut_forward_reads.qza


UUID:        3638611d-1767-413b-9390-70ee3d78e4ff
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [10]:
!qiime demux summarize \
  --i-data $data_dir/fungut_forward_reads.qza \
  --o-visualization $data_dir/demux_summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: livia_data/demux_summary.qzv


# Denoising

In [12]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/rep_seqs.qza \
    --o-table $data_dir/table.qza \
    --o-denoising-stats $data_dir/stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: livia_data/table.qza
Saved FeatureData[Sequence] to: livia_data/rep_seqs.qza
Saved SampleData[DADA2Stats] to: livia_data/stats.qza


In [13]:
!qiime feature-table summarize \
  --i-table $data_dir/table.qza \
  --o-visualization $data_dir/table_summary.qzv \
--m-sample-metadata-file $data_dir/fungut_metadata.tsv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: livia_data/table_summary.qzv


In [14]:
! qiime feature-table tabulate-seqs \
  --i-data $data_dir/rep_seqs.qza \
  --o-visualization $data_dir/rep_seqs.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: livia_data/rep_seqs.qzv


In [6]:
Visualization.load(f"{data_dir}/rep_seqs.qzv")

<visualization: Visualization uuid: a5a348e8-bc44-4076-96e2-00a801352821>

# Taxonomy

In [ ]:
qiime rescript filter-seqs-length-by-taxon \
  --i-sequences $data_dir/its-refseqs-unfiltered.qza \
  --i-taxonomy $data_dir/its-taxonomy-unfiltered.qza \
  --p-labels Fungi \
  --p-min-lens 150 \
  --o-filtered-seqs $data_dir/its-refseqs-filtered.qza \
  --o-discarded-seqs $data_dir/its-refseqs-discarded.qza